#  Tradutor de Textos - Implantação
## Utilização do modelo [MarianMT](https://huggingface.co/transformers/model_doc/marian.html) para tradução. 


* Este componente faz tradução multilingual entre uma série de idiomas, tais como Português, Inglês, Espanhol, Alemão, Holandês, Japonês, Chinês e outros. 
* Para adaptar para traduções em outros idiomas é necessário verificacar se há o modelo pré treinado disponível no MarianMT e adicionar o par de tradução (idioma fonte, idioma alvo) no arquivo ```translator.py``` da seguinte forma:
    
    ```python
    # Par de tradução e respectivo modelo
    MODEL_MAP = {
        ...
        ('português', 'inglês'): 'Helsinki-NLP/opus-mt-ROMANCE-en',
        ...
    }

    ...

    # Código dos idiomas inseridos (se não houver)
    LANGUAGE_PREFIX = {
        ...
        'português': 'pt',
        'inglês': 'en',
        ...
    }
    ...
    ```

* As métricas computadas são o BLEU e o ROUGE-L 


### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [1]:
%%writefile Model.py

import joblib
import numpy as np
import pandas as pd
from translator import Translator

class Model:
    def __init__(self):
        self.loaded = False

    def load(self):
        
        # Load artifacts
        artifacts = joblib.load("/tmp/data/translator.joblib")
        self.model_parameters = artifacts["model_parameters"]
        self.inference_parameters = artifacts["inference_parameters"]
        
        # Initialize translator
        self.translator = Translator(**self.model_parameters)
        
        # Set model loaded
        self.loaded = True
        print("Loaded model")
    
    def class_names(self):
        column_names = list(self.inference_parameters['columns']) + [self.inference_parameters['output_column_name']]
        return column_names

    def predict(self, X, feature_names, meta=None):
        if not self.loaded:
            self.load()
            
        if feature_names:
            # Antes de utilizar o conjunto de dados X no modelo, reordena suas features de acordo com a ordem utilizada no treinamento
            df = pd.DataFrame(X, columns=feature_names)
            X = df[self.inference_parameters['columns']]
        
        else:
            X = pd.DataFrame(X, columns=self.inference_parameters['columns'])
            
        # Translate sentences
        translated_texts = self.translator(X[self.inference_parameters['text_column_name']])
        
        # Output dataframe
        output_df = X.copy()
        output_df[self.inference_parameters['output_column_name']] = translated_texts
        
        # Output
        output = output_df.to_numpy()

        return output

Writing Model.py
